In [ ]:
!pip install pennylane pandas tensorflow scikit-learn matplotlib seaborn

In [ ]:
import torchvision
print(torchvision.__version__)

In [ ]:
import os
import time
import json
import logging
import itertools
from pathlib import Path
import pennylane as qml
from pennylane import numpy as np
import autograd.numpy as anp
import pandas as pd
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import f1_score
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from pennylane.transforms import add_noise
from pennylane.noise import NoiseModel, op_in

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')

# ==============================================================================
# 1. PENGATURAN AWAL: DIREKTORI, LOGGING & KONFIGURASI GLOBAL
# ==============================================================================

Path("results").mkdir(exist_ok=True)
Path("results/plots").mkdir(exist_ok=True)
LOG_FILE = Path("results/experiment_log.csv")
PLOT_DIR = Path("results/plots")

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s", handlers=[logging.StreamHandler()], force=True)

# ==============================================================================
# 2. KOMPONEN QCNN (UNITARY, EMBEDDING, SIRKUIT)
# ==============================================================================

def U_TTN(params, wires):
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])

def U_5(params, wires):
    qml.RX(params[0], wires=wires[0]); qml.RX(params[1], wires=wires[1])
    qml.RZ(params[2], wires=wires[0]); qml.RZ(params[3], wires=wires[1])
    qml.CRZ(params[4], wires=[wires[1], wires[0]])
    qml.CRZ(params[5], wires=[wires[0], wires[1]])
    qml.RX(params[6], wires=wires[0]); qml.RX(params[7], wires=wires[1])
    qml.RZ(params[8], wires=wires[0]); qml.RZ(params[9], wires=wires[1])

def U_6(params, wires):
    qml.RX(params[0], wires=wires[0]); qml.RX(params[1], wires=wires[1])
    qml.RZ(params[2], wires=wires[0]); qml.RZ(params[3], wires=wires[1])
    qml.CRX(params[4], wires=[wires[1], wires[0]])
    qml.CRX(params[5], wires=[wires[0], wires[1]])
    qml.RX(params[6], wires=wires[0]); qml.RX(params[7], wires=wires[1])
    qml.RZ(params[8], wires=wires[0]); qml.RZ(params[9], wires=wires[1])

def U_9(params, wires):
    qml.Hadamard(wires=wires[0]); qml.Hadamard(wires=wires[1])
    qml.CZ(wires=[wires[0], wires[1]])
    qml.RX(params[0], wires=wires[0]); qml.RX(params[1], wires=wires[1])

def U_13(params, wires):
    qml.RY(params[0], wires=wires[0]); qml.RY(params[1], wires=wires[1])
    qml.CRZ(params[2], wires=[wires[1], wires[0]])
    qml.RY(params[3], wires=wires[0]); qml.RY(params[4], wires=wires[1])
    qml.CRZ(params[5], wires=[wires[0], wires[1]])

def U_14(params, wires):
    qml.RY(params[0], wires=wires[0]); qml.RY(params[1], wires=wires[1])
    qml.CRX(params[2], wires=[wires[1], wires[0]])
    qml.RY(params[3], wires=wires[0]); qml.RY(params[4], wires=wires[1])
    qml.CRX(params[5], wires=[wires[0], wires[1]])

def U_15(params, wires):
    qml.RY(params[0], wires=wires[0]); qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[2], wires=wires[0]); qml.RY(params[3], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])

def U_SO4(params, wires):
    qml.RY(params[0], wires=wires[0]); qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[2], wires=wires[0]); qml.RY(params[3], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[4], wires=wires[0]); qml.RY(params[5], wires=wires[1])

def U_SU4(params, wires):
    qml.U3(params[0], params[1], params[2], wires=wires[0])
    qml.U3(params[3], params[4], params[5], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[6], wires=wires[0]); qml.RZ(params[7], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[8], wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.U3(params[9], params[10], params[11], wires=wires[0])
    qml.U3(params[12], params[13], params[14], wires=wires[1])
    
def U_HS(params, wires):
    qml.PauliZ(wires=wires[0]); qml.RY(params[0], wires=wires[1])
    qml.CZ(wires=[wires[0], wires[1]])
    qml.CRX(params[0], wires=[wires[1], wires[0]])
    qml.RY(params[1], wires=wires[1])

UNITARY_GATES = {'U_TTN': {'fn': U_TTN, 'params': 2}, 'U_5': {'fn': U_5, 'params': 10}, 'U_6': {'fn': U_6, 'params': 10}, 'U_9': {'fn': U_9, 'params': 2}, 'U_13': {'fn': U_13, 'params': 6}, 'U_14': {'fn': U_14, 'params': 6}, 'U_15': {'fn': U_15, 'params': 4}, 'U_SO4': {'fn': U_SO4, 'params': 6}, 'U_SU4': {'fn': U_SU4, 'params': 15}, 'U_HS': {'fn': U_HS, 'params': 2}}

def data_embedding(X, embedding_type='amplitude'):
    n_qubits = 8
    if embedding_type == 'amplitude':
        qml.AmplitudeEmbedding(X, wires=range(n_qubits), normalize=True, pad_with=0.)
    elif embedding_type == 'qubit':
        qml.AngleEmbedding(X, wires=range(n_qubits), rotation='Y')
    elif embedding_type == 'dense':
        qml.AngleEmbedding(X[:n_qubits], wires=range(n_qubits), rotation='X')
        qml.AngleEmbedding(X[n_qubits:2*n_qubits], wires=range(n_qubits), rotation='Y')
    else:
        raise ValueError(f"Unknown embedding type: {embedding_type}")

def conv_layer(U, params, wires_list):
    for wires in wires_list:
        U(params, wires=wires)

def pooling_layer(V, params, wires_list):
    for i, wires in enumerate(wires_list):
        V(params[i*2:(i+1)*2], wires=wires)

def Pooling_ansatz1(params, wires):
    qml.CRZ(params[0], wires=[wires[0], wires[1]])
    qml.PauliX(wires=wires[0])
    qml.CRX(params[1], wires=[wires[0], wires[1]])

def qcnn_structure(U_fn, params, U_params, num_classes):
    num_measurement_qubits = np.ceil(np.log2(num_classes))
    param_idx = 0
    conv_layer(U_fn, params[param_idx:param_idx + U_params], [[0, 1], [2, 3], [4, 5], [6, 7]])
    param_idx += U_params
    pooling_layer(Pooling_ansatz1, params[param_idx:param_idx + 8], [[1, 0], [3, 2], [5, 4], [7, 6]])
    param_idx += 8
    active_wires = [0, 2, 4, 6]
    if num_measurement_qubits >= 3: return active_wires[:int(num_measurement_qubits)]
    conv_layer(U_fn, params[param_idx:param_idx + U_params], [[0, 2], [4, 6]])
    param_idx += U_params
    pooling_layer(Pooling_ansatz1, params[param_idx:param_idx + 4], [[2, 0], [6, 4]])
    param_idx += 4
    active_wires = [0, 4]
    if num_measurement_qubits == 2: return active_wires
    conv_layer(U_fn, params[param_idx:param_idx + U_params], [[0, 4]])
    param_idx += U_params
    pooling_layer(Pooling_ansatz1, params[param_idx:param_idx + 2], [[4, 0]])
    return [0]

# ==============================================================================
# 3. INFRASTRUKTUR: DATA, MODEL, TRAINING, DAN EVALUASI
# ==============================================================================

def get_device(n_wires, has_noise):
    if not has_noise:
        return qml.device("default.qubit", wires=n_wires)
    else:
        return qml.device("default.mixed", wires=n_wires)

def build_realistic_noise_model(noise_params):
    """Membangun NoiseModel PennyLane dari dictionary parameter noise."""
    # Ekstrak parameter noise dengan nilai default 0 atau None
    p_single = noise_params.get('depolarizing_single', 0)
    p_two = noise_params.get('depolarizing_two', 0)
    t1 = noise_params.get('t1', None)  # dalam nanodetik
    t2 = noise_params.get('t2', None)  # dalam nanodetik

    if p_single == 0 and p_two == 0 and t1 is None:
        return None

    noise_map = {}

    # --- Mendefinisikan Fungsi-fungsi Operasi Noise ---
    def single_qubit_depolarizing(op, **kwargs):
        # Menerapkan noise depolarisasi setelah gerbang satu-qubit.
        return [qml.DepolarizingChannel(p_single, wires=w) for w in op.wires] if p_single > 0 else []

    def two_qubit_depolarizing(op, **kwargs):
        # Menerapkan noise depolarisasi setelah gerbang dua-qubit.
        return [qml.DepolarizingChannel(p_two, wires=w) for w in op.wires] if p_two > 0 else []

    def thermal_relaxation_single(op, **kwargs):
        # Mensimulasikan dekoherensi (kehilangan informasi) selama gerbang satu-qubit beroperasi.
        return [qml.ThermalRelaxationError(0, t1, t2, GATE_TIMES['single_qubit'], wires=w) for w in op.wires] if t1 and t2 else []

    def thermal_relaxation_two(op, **kwargs):
        # Mensimulasikan dekoherensi selama gerbang dua-qubit beroperasi.
        return [qml.ThermalRelaxationError(0, t1, t2, GATE_TIMES['two_qubit'], wires=w) for w in op.wires] if t1 and t2 else []

    # --- Memetakan Noise ke Jenis Gerbang ---
    SINGLE_QUBIT_OPS = op_in([qml.RX, qml.RY, qml.RZ, qml.Hadamard, qml.PauliX, qml.U3])
    TWO_QUBIT_OPS = op_in([qml.CNOT, qml.CZ, qml.CRX, qml.CRZ])
    
    # Menggabungkan noise depolarisasi dan relaksasi termal untuk setiap jenis gerbang
    noise_map[SINGLE_QUBIT_OPS] = lambda op, **kwargs: single_qubit_depolarizing(op, **kwargs) + thermal_relaxation_single(op, **kwargs)
    noise_map[TWO_QUBIT_OPS] = lambda op, **kwargs: two_qubit_depolarizing(op, **kwargs) + thermal_relaxation_two(op, **kwargs)

    return NoiseModel(noise_map)


def load_and_preprocess_data(dataset_name, classes):
    logging.info(f"Memuat dan memproses dataset: {dataset_name} untuk kelas {classes}")
    if dataset_name in ["mnist", "fashion_mnist"]:
        (x_train, y_train), (x_test, y_test) = getattr(tf.keras.datasets, dataset_name).load_data()
        x_train = tf.image.resize(tf.convert_to_tensor(x_train, dtype=tf.float32)[..., tf.newaxis], (32, 32)) / 255.0
        x_test = tf.image.resize(tf.convert_to_tensor(x_test, dtype=tf.float32)[..., tf.newaxis], (32, 32)) / 255.0
    elif dataset_name == "cifar10":
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
        x_train_float = tf.cast(x_train, tf.float32)
        x_test_float = tf.cast(x_test, tf.float32)
        x_train, x_test = tf.image.rgb_to_grayscale(x_train_float) / 255.0, tf.image.rgb_to_grayscale(x_test_float) / 255.0
    else:
        raise ValueError(f"Unsupported dataset: {dataset_name}")
    train_filter = np.isin(y_train.flatten(), classes)
    test_filter = np.isin(y_test.flatten(), classes)
    X_train, Y_train = x_train.numpy()[train_filter], y_train[train_filter]
    X_test, Y_test = x_test.numpy()[test_filter], y_test[test_filter]
    class_map = {old_label: new_label for new_label, old_label in enumerate(classes)}
    Y_train = np.array([class_map[int(y)] for y in Y_train.flatten()])
    Y_test = np.array([class_map[int(y)] for y in Y_test.flatten()])
    return X_train, Y_train, X_test, Y_test

def apply_dimensionality_reduction(X_train, Y_train, X_test, method, embedding_type):
    if embedding_type == 'amplitude': target_dim = 256
    elif embedding_type == 'dense': target_dim = 16
    else: target_dim = 8
    logging.info(f"Menerapkan '{method}' untuk data embedding '{embedding_type}'.")
    if method == 'bilinear_resize':
        if embedding_type != 'amplitude':
            raise ValueError(f"Bilinear resizing hanya untuk 'amplitude' embedding, bukan '{embedding_type}'.")
    
        # 2. PROCEED WITH RESIZING
        try:
            target_size = int(np.sqrt(target_dim))
            
            X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
            X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
            
            X_train_resized = tf.image.resize(X_train_tensor, (target_size, target_size), method='bilinear')
            X_test_resized = tf.image.resize(X_test_tensor, (target_size, target_size), method='bilinear')
            
            X_train_reduced = tf.reshape(X_train_resized, (X_train_resized.shape[0], -1)).numpy()
            X_test_reduced = tf.reshape(X_test_resized, (X_test_resized.shape[0], -1)).numpy()
    
            return X_train_reduced, X_test_reduced
        except Exception as e:
            raise e
    else:
        X_train_flat = X_train.reshape(X_train.shape[0], -1)
        X_test_flat = X_test.reshape(X_test.shape[0], -1)
        if method == 'pca':
            reducer = PCA(n_components=target_dim)
            X_train_reduced = reducer.fit_transform(X_train_flat)
            X_test_reduced = reducer.transform(X_test_flat)
        elif method == 'autoencoder':
            input_dim = X_train_flat.shape[1]
            encoder_input = layers.Input(shape=(input_dim,), name="encoder_input")
            encoded_layer = layers.Dense(target_dim, activation='relu')(encoder_input)
            encoder = Model(encoder_input, encoded_layer, name="encoder")
            decoder_input = layers.Input(shape=(target_dim,), name="decoder_input")
            decoded_layer = layers.Dense(input_dim, activation='sigmoid')(decoder_input)
            decoder = Model(decoder_input, decoded_layer, name="decoder")
            autoencoder = Model(encoder_input, decoder(encoder.output), name="autoencoder")
            autoencoder.compile(optimizer='adam', loss='mse')
            autoencoder.fit(X_train_flat, X_train_flat, epochs=20, batch_size=32, shuffle=True, verbose=0)
            X_train_reduced, X_test_reduced = encoder.predict(X_train_flat, verbose=0), encoder.predict(X_test_flat, verbose=0)
        else: raise ValueError(f"Metode reduksi/persiapan tidak dikenal: {method}")
    min_val, max_val = X_train_reduced.min(), X_train_reduced.max()
    X_train_scaled = (X_train_reduced - min_val) * (np.pi / (max_val - min_val + 1e-9))
    test_min, test_max = X_test_reduced.min(), X_test_reduced.max()
    X_test_scaled = (X_test_reduced - test_min) * (np.pi / (test_max - test_min + 1e-9))
    return X_train_scaled, X_test_scaled

# --- Komponen infrastruktur lainnya (Optimizer, Metrik, Plotting) tidak berubah ---
def get_optimizer(name, lr, steps=80):
    if name.lower() == 'adam': return qml.AdamOptimizer(stepsize=lr)
    if name.lower() == 'sgd': return qml.GradientDescentOptimizer(stepsize=lr)
    if name.lower() == 'nesterovmomentum': return qml.NesterovMomentumOptimizer(stepsize=lr)
    if name.lower() == 'rmsprop': return qml.RMSPropOptimizer(stepsize=lr)
    if name.lower() == 'spsa': return qml.SPSAOptimizer(maxiter=steps)
    raise ValueError(f"Unknown optimizer: {name}")

def calculate_metrics(labels, predictions, cost_fn_type):
    if not len(labels): return 0.0, 0.0
    pred_labels = [1 if p > 0 else 0 for p in predictions] if cost_fn_type in ('mse', 'mae') else np.argmax(predictions, axis=1)
    return np.sum(pred_labels == labels) / len(labels), f1_score(labels, pred_labels, average='weighted', zero_division=0)

def plot_history(history, save_path):
    plt.style.use("seaborn-v0_8-whitegrid")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    ax1.plot(history['cost'], label='Training Loss')
    ax1.set(title="Training Loss vs. Iterasi", xlabel="Iterasi", ylabel="Loss")
    ax1.legend()
    iterations = range(2, len(history['train_acc']) * 2 + 1, 2)
    ax2.plot(iterations, history['train_acc'], label='Akurasi Training Batch', marker='o', linestyle='--')
    ax2.set(title="Akurasi Training Batch vs. Iterasi", xlabel="Iterasi", ylabel="Akurasi")
    ax2.legend()
    fig.tight_layout()
    plt.savefig(save_path)
    plt.close(fig)

# --- Fungsi Inti Eksperimen yang Diperbarui ---
def run_single_variation(variation):
    """Menjalankan satu siklus training dan evaluasi penuh untuk satu variasi."""
    start_time = time.time()
    # Ekstrak dictionary parameter noise dari variasi
    noise_params = variation.get('noise_params', {})
    has_noise = bool(noise_params)

    # Langkah 1: Inisialisasi Device Kuantum
    device = get_device(n_wires=8, has_noise=has_noise)
    
    # Langkah 2: Definisi QNode Dasar (menangani error pembacaan/SPAM secara internal)
    @qml.qnode(device, interface="autograd")
    def base_quantum_circuit(X, params):
        unitary_info = UNITARY_GATES[variation['conv_ansatz']]
        data_embedding(X, embedding_type=variation['embedding'])
        measurement_wires = qcnn_structure(unitary_info['fn'], params, unitary_info['params'], len(variation['classes']))

        readout_error = noise_params.get('readout_error', 0)
        if readout_error > 0:
            # Modelkan error pembacaan simetris dengan BitFlip.
            # Ini akan membalik state qubit (0->1 atau 1->0) dengan probabilitas `readout_error`.
            for wire in measurement_wires:
                qml.BitFlip(readout_error, wires=wire)
        
        is_binary = variation['cost_fn'] in ('mse', 'mae')
        return qml.expval(qml.PauliZ(measurement_wires[0])) if is_binary else qml.probs(wires=measurement_wires)

    # Langkah 3: Bangun sirkuit akhir (bisa ideal atau dengan noise)
    final_circuit = base_quantum_circuit
    noise_model = build_realistic_noise_model(noise_params)
    if noise_model:
        final_circuit = add_noise(base_quantum_circuit, noise_model)
        logging.info("Model noise realistis telah diterapkan pada sirkuit.")

    # Langkah 4: Definisi Fungsi Biaya
    def square_loss(labels, predictions):
        labels_mapped = anp.array([1 if l == 1 else -1 for l in labels])
        return anp.mean((labels_mapped - anp.array(predictions)) ** 2)
        
    def multiclass_cross_entropy(labels, predictions): loss = 0; [loss := loss - anp.log(anp.clip(p, 1e-9, 1 - 1e-9)[l]) for l, p in zip(labels, predictions)]; return loss / len(labels)

    def cost_fn(params, X, Y):
        predictions = []
    
        for i, x in enumerate(X):
            pred = final_circuit(x, params)
            current_shape = np.shape(pred)
            predictions.append(pred)
    
        if (variation['cost_fn'] == 'mse' or variation['cost_fn'] == 'mae'):
            return square_loss(Y, predictions)
        else:
            return multiclass_cross_entropy(Y, predictions)

    # Langkah 6: Muat dan proses data
    X_train, Y_train, X_test, Y_test = load_and_preprocess_data(variation['dataset'], variation['classes'])
    X_train, X_test = apply_dimensionality_reduction(X_train, Y_train, X_test, variation['reduction'], variation['embedding'])

    # Langkah 7: Inisialisasi model dan optimizer
    unitary_info, num_classes = UNITARY_GATES[variation['conv_ansatz']], len(variation['classes'])
    num_measurement_qubits = np.ceil(np.log2(num_classes))
    total_params = unitary_info['params'] + 8
    if num_measurement_qubits < 3: total_params += unitary_info['params'] + 4
    if num_measurement_qubits < 2: total_params += unitary_info['params'] + 2
    params = np.random.uniform(0, 2 * np.pi, total_params, requires_grad=True)
    opt = get_optimizer(variation['optimizer'], variation['learning_rate'], variation['steps'])

    # Langkah 8: Loop Training
    history = {'cost': [], 'train_acc': []}
    for it in range(variation['steps']):
        batch_indices = np.random.randint(0, len(X_train), (variation['batch_size'],))
        X_batch, Y_batch = X_train[batch_indices], Y_train[batch_indices]
        params, cost_val = opt.step_and_cost(lambda p: cost_fn(p, X_batch, Y_batch), params)
        history['cost'].append(cost_val)

        if (it + 1) % 2 == 0:
            preds_train_batch = [final_circuit(x, params) for x in X_batch]
            train_acc, _ = calculate_metrics(Y_batch, preds_train_batch, variation['cost_fn'])
            history['train_acc'].append(train_acc)
            logging.info(f"Iter: {it+1:4d} | Cost: {cost_val:.4f} | Akurasi Batch: {train_acc:.4f}")

    # Langkah 9: Evaluasi Akhir dan Logging
    final_preds_test = [final_circuit(x, params) for x in X_test]
    test_acc, test_f1 = calculate_metrics(Y_test, final_preds_test, variation['cost_fn'])
    logging.info(f"Akurasi Test: {test_acc:.4f} | F1-score Test: {test_f1:.4f}")

    variation_id_parts = {k: v for k, v in variation.items() if k != 'noise_params'}
    variation_id = "-".join(map(str, variation_id_parts.values()))
    plot_path = PLOT_DIR / f"{variation_id}.png"
    plot_history(history, plot_path)

    return {
        "variation_id": variation_id,
        "train_time_sec": round(time.time() - start_time, 2),
        "test_acc": round(test_acc.item(), 4), "test_f1": round(test_f1.item(), 4),
        "plot_path": str(plot_path), **variation_id_parts
    }

# ==============================================================================
# 4. MANAJEMEN EKSPERIMEN
# ==============================================================================
def generate_variations(grid):
    """Menghasilkan daftar semua variasi eksperimen dari sebuah grid."""
    reps = grid.pop('repetition', 1)
    noise_configs = grid.pop('noise_configs', {'noiseless': {}})

    keys, values = zip(*grid.items())
    base_variations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    combined_variations = []
    for var in base_variations:
        for name, params in noise_configs.items():
            new_var = var.copy()
            new_var['noise_config_name'] = name 
            new_var['noise_params'] = params  
            combined_variations.append(new_var)

    all_variations = []
    for var in combined_variations:
        for i in range(1, reps + 1):
            rep_var = var.copy()
            rep_var['repetition'] = i
            all_variations.append(rep_var)

    def is_valid(v):
        if v.get('reduction') == 'bilinear_resize' and v.get('embedding') != 'amplitude': return False
        if v.get('cost_fn') in ('mse', 'mae') and len(v.get('classes', [])) > 2: return False
        return True

    valid_variations = [v for v in all_variations if is_valid(v)]
    grid['repetition'] = reps
    grid['noise_configs'] = noise_configs
    return valid_variations

def load_or_initialize_logfile():
    if LOG_FILE.exists():
        logging.info(f"File log ditemukan: {LOG_FILE}")
        return pd.read_csv(LOG_FILE)
    logging.info("File log tidak ditemukan. Membuat file baru.")
    return pd.DataFrame()

def get_remaining_variations(all_variations, log_df):
    if log_df.empty: return all_variations
    completed_ids = set(log_df['variation_id'].dropna())
    
    def get_variation_id(v):
        id_parts = {k: val for k, val in v.items() if k != 'noise_params'}
        return "-".join(map(str, id_parts.values()))
        
    return [v for v in all_variations if get_variation_id(v) not in completed_ids]

def run_experiment_suite(grid):
    all_variations = generate_variations(grid)
    log_df = load_or_initialize_logfile()
    remaining_variations = get_remaining_variations(all_variations, log_df)

    if not remaining_variations:
        logging.info("Semua variasi eksperimen telah selesai. Selesai.")
        return

    logging.info(f"Total variasi: {len(all_variations)} | Selesai: {len(all_variations) - len(remaining_variations)} | Sisa: {len(remaining_variations)}")

    for i, variation in enumerate(remaining_variations):
        log_info = {k:v for k,v in variation.items() if k != 'noise_params'}
        logging.info(f"\n--- Memulai Variasi {i+1}/{len(remaining_variations)} ---\n{json.dumps(log_info, indent=2)}")
        try:
            result = run_single_variation(variation)
            log_df = pd.concat([log_df, pd.DataFrame([result])], ignore_index=True)
            log_df.to_csv(LOG_FILE, index=False)
            logging.info(f"--- Variasi {i+1}/{len(remaining_variations)} Selesai. Hasil dicatat di {LOG_FILE} ---")
        except Exception as e:
            logging.error(f"Variasi gagal: {variation['noise_config_name']}\nError: {e}", exc_info=True)
            id_parts = {k: v for k, v in variation.items() if k != 'noise_params'}
            failed_result = {"variation_id": "-".join(map(str, id_parts.values())), "train_time_sec": "FAILED", **id_parts}
            log_df = pd.concat([log_df, pd.DataFrame([failed_result])], ignore_index=True)
            log_df.to_csv(LOG_FILE, index=False)


In [ ]:
EXPERIMENT_GRID = {
        # "dataset": ["imagenette", "mnist", "fashion_mnist", "cifar10"],
        "dataset": ["mnist"],
        "classes":  [[0, 1]], #mnist
        # "classes":  [[0, 1, 2, 3]], #mnist mul
        # "classes":  [[7, 9]], #fmnist
        # "classes":  [[0, 1, 8, 9]], #fmnist mul
        # "classes":  [[3, 8]], #cifar
        "reduction": [
                        "bilinear_resize",
                        "autoencoder", 
                        "pca",
                     ],
        "embedding": ["amplitude", "dense", "qubit"],
        # "embedding": ["dense"],
        "conv_ansatz": [
            #------------- ANZ 1
            'U_TTN', 'U_5',
            #------------- ANZ 2
            'U_9', 'U_13', 
            # ------------ ANZ 3
            'U_14', 'U_15', 
            # ------------ ANZ 4
            'U_SO4', 'U_SU4',
            # ------------ ANZ 5
            'U_6', 'U_HS',
        ],
        "optimizer": [
            "Adam",
            
            "RMSProp",
            "SPSA", 
            
            "SGD", 
            "NesterovMomentum",
        ],
        "cost_fn": ["mse", "mae", "cross_entropy", ],
        'learning_rate': [0.01],
        'batch_size': [32],
        'steps': [80],
        'repetition': 1,
    
        # --- CONFIGURABLE NOISE MODELS ---
        'noise_configs': {
            'noiseless': {},
            'realistic_nisq': {
                't1': 150000,                  # T1 relaxation time in nanoseconds (150 µs)
                't2': 100000,                  # T2 dephasing time in nanoseconds (100 µs)
                'depolarizing_single': 0.0005, # 0.05% error per single-qubit gate
                'depolarizing_two': 0.005,     # 0.5% error per two-qubit gate
                'readout_error': 0.01          # 1% SPAM error probability
            },
            # 'high_depolarizing': {
            #     'depolarizing_single': 0.005,
            #     'depolarizing_two': 0.05,
            #     'readout_error': 0.02
            # }
        }
    }
    
run_experiment_suite(EXPERIMENT_GRID)

GATE_TIMES = {
    'single_qubit': 50,  # Waktu untuk gerbang seperti U3, RY, RX, RZ, H
    'two_qubit': 300     # Waktu untuk gerbang seperti CNOT, CRX, CRZ, CZ
}

run_experiment_suite(EXPERIMENT_GRID)